In [8]:
import pandas as pd
import numpy as np
import pickle
# from scipy.special import expit

In [4]:
defs = pd.read_csv("db/sample/clean/def_col_vectors.csv", names=["word", "word_vector","definition", "def_vectors"], sep=":", index_col=None, keep_default_na=False, na_values=[""])
word_list = pd.read_csv("db/sample/clean/word_col_vectors.csv", names=["word", "word_vector"], sep=":", index_col=None, keep_default_na=False, na_values=[""])


X = []
y = []

defs = defs.values.tolist()
word_list = word_list.values.tolist()
defs_main = defs[:6575]

defs_main = [[str(d[0]).strip(), d[1], str(d[2]).strip(), str(d[3]).strip()] for d in defs_main]
size = len(word_list)

for n in defs_main:
    main_word = str(n[0])
    definition = str(n[2]).split()
    vector = str(n[3]).split()
    out = np.zeros((size,), dtype=int)
    inp = np.zeros((size,), dtype=int)
    for word in word_list:
        if word[0] is main_word:
            out[word[1]] = 1
        if word[0] in definition:
            inp[word[1]] = 1
    X.append(inp)
    y.append(out)

    
X = np.asarray(X)
y = np.asarray(y)


#### check if arrays contain desired value

In [1]:
def check_if_found(array, val):
    flag = False
    for arr in array:
        for a in arr:
            if a != val:
                print(a)
                print("found")
                flag = True
                break
        if flag:
            break
    return

In [5]:
check_if_found(X,0)
check_if_found(y,0)

found
found


In [6]:
def sigmoid(x):
    return np.around(1/(1+np.exp(-x)), decimals=30)

def der_sigmoid(x):
    return 1*(1-x)

epoch=20
lr=0.1

input_layer_neurons = X.shape[1] # 3
hiddenlayer_neurons = 3
output_neurons = size

wh = np.random.uniform(size = (input_layer_neurons, hiddenlayer_neurons))
bh = np.random.uniform(size = (1, hiddenlayer_neurons))
wout = np.random.uniform(size = (hiddenlayer_neurons, output_neurons))
bout = np.random.uniform(size = (1, output_neurons))

for i in range(epoch):
    # Forward propagation
    hidden_layer_input1 = np.dot(X,wh)
    hidden_layer_input = hidden_layer_input1 + bh
    hidden_layer_activations = sigmoid(hidden_layer_input)
    output_layer_input1 = np.dot(hidden_layer_activations, wout)
    output_layer_input = output_layer_input1 + bout
    output = sigmoid(output_layer_input)

    # Backpropagation
    E = y-output
    slope_output_layer = der_sigmoid(output)
    slope_hidden_layer = der_sigmoid(hidden_layer_activations)
    d_output = E * slope_output_layer
    Error_at_hidden_layer = d_output.dot(wout.T)
    d_hiddenlayer = Error_at_hidden_layer * slope_hidden_layer
    wout += hidden_layer_activations.T.dot(d_output) * lr
    bout += np.sum(d_output, axis=0, keepdims = True) * lr
    wh += X.T.dot(d_hiddenlayer) * lr
    bh += np.sum(d_hiddenlayer, axis=0, keepdims = True) * lr



C:\Users\ritiksax\AppData\Local\Continuum\miniconda3\envs\revdict\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


#### save all data

In [10]:
with open("pkl-db/wh.pkl","wb") as f:
    pickle.dump(wh,f)

with open("pkl-db/bh.pkl","wb") as f:
    pickle.dump(bh,f)

with open("pkl-db/wout.pkl","wb") as f:
    pickle.dump(wout,f)

with open("pkl-db/bout.pkl","wb") as f:
    pickle.dump(bout,f)

with open("pkl-db/output.pkl","wb") as f:
    pickle.dump(output,f)

with open("pkl-db/X.pkl","wb") as f:
    pickle.dump(X,f)

with open("pkl-db/y.pkl","wb") as f:
    pickle.dump(y,f)


#### check if data is saved correctly or not

In [13]:
with open("pkl-db/wh.pkl","rb") as f:
    whpkl = pickle.load(f)
    check_if_found(whpkl,0)
    
with open("pkl-db/bh.pkl","rb") as f:
    bhpkl = pickle.load(f)
    check_if_found(bhpkl,0)
    
with open("pkl-db/wout.pkl","rb") as f:
    woutpkl = pickle.load(f)
    check_if_found(woutpkl,0)
    
with open("pkl-db/bout.pkl","rb") as f:
    boutpkl = pickle.load(f)
    check_if_found(boutpkl,0)
    
with open("pkl-db/output.pkl","rb") as f:
    outputpkl = pickle.load(f)
    check_if_found(outputpkl,0.5)
    
with open("pkl-db/X.pkl","rb") as f:
    Xpkl = pickle.load(f)
    check_if_found(Xpkl,0)
    
with open("pkl-db/y.pkl","rb") as f:
    ypkl = pickle.load(f)
    check_if_found(ypkl,0)

0.9346699565064474
found
-18879.739124911714
found
-38.465961730243265
found
-40.034687716883006
found
4.1e-18
found
1
found
1
found
